In [1]:
import time
import pandas as pd

In [2]:
ROOT = '/data1/wgs_APA/validation_res'

In [7]:
gem_1 = pd.read_csv(ROOT + '/gem0/protospacer_calls_per_cell.csv')
gem_1.head()

,cell_barcode,num_features,feature_call,num_umis
0,AAATGGAGTGGCGCTT-1,1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,1,Ascl2-2,27


In [8]:
from geomux import Geomux, read_table

gem_1['feature_call'] = gem_1['feature_call'].str.split('|')
gem_1['num_umis'] = gem_1['num_umis'].str.split('|')
gem_1 = gem_1.explode(list(['feature_call','num_umis']))
gem_1.drop(columns='num_features', inplace=True)
gem_1.rename(columns={"cell_barcode": "barcode", "feature_call": "guide", "num_umis": "n_umi"}, inplace=True)
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [16]:
# remove NA from matrix
gem_1 = gem_1[gem_1['guide'].notna()]
gem_1 = gem_1[gem_1['n_umi'].notna()]
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [23]:
# conver to cell by guide matrix, with n_umi as value, with NA as 0
gem_1_mat = gem_1.pivot(index='barcode', columns='guide', values='n_umi')
gem_1_mat.fillna(0, inplace=True)
gem_1_mat
# remove index and column name
gem_1_mat.index.name = None
gem_1_mat.columns.name = None
gem_1_mat

,Ascl2-2,Ascl2-3,Ascl2-4,BIRC2-2,BIRC2-4,BRIP1-2,BRIP1-4,CEP44-1,CEP44-2,CEP44-4,...,ZNF527-2,ZNF527-4,ZNF530-1,ZNF530-2,ZNF530-3,ZNF530-4,negative_control-1,negative_control-2,znf623-2,znf623-4
AAACCCAAGACGCTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGCTAATCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGACTGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGGAGGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTGCGTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAAGGACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCAGGAACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,42,0,0,0,0,0,0
TTTGTTGTCCAAAGGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCCCTCATG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,141,0,0,0,0,0,0


In [30]:
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [32]:
# set dtype of columns in gem_1, to dtype={"barcode": str, "guide": str, "n_umi": int},
gem_1 = gem_1.astype({"barcode": str, "guide": str, "n_umi": int})
gem_1

,barcode,guide,n_umi
0,AAATGGAGTGGCGCTT-1,Ascl2-2,155
1,AACAAAGTCCCGATCT-1,Ascl2-2,197
2,AACCATGCAAATTGCC-1,Ascl2-2,112
3,AACCATGGTCAACCTA-1,Ascl2-2,37
4,AACGGGAGTTAGTCGT-1,Ascl2-2,27
...,...,...,...
25132,CAATCGAGTCTAGTGT-1,ZNF345-4,215
25133,TCAAGACAGTGGATAT-1,znf623-4,15
25133,TCAAGACAGTGGATAT-1,ZNF345-4,75
25134,TTGCTGCCAGGGCTTC-1,znf623-4,12


In [33]:
matrix = gem_1.pivot_table(
    index="barcode", columns="guide", values="n_umi", fill_value=0
)
matrix

guide,Ascl2-2,Ascl2-3,Ascl2-4,BIRC2-2,BIRC2-4,BRIP1-2,BRIP1-4,CEP44-1,CEP44-2,CEP44-4,...,ZNF527-2,ZNF527-4,ZNF530-1,ZNF530-2,ZNF530-3,ZNF530-4,negative_control-1,negative_control-2,znf623-2,znf623-4
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGACGCTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGCTAATCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGACTGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGGGAGGGT-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGTGCGTCC-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCAAGGACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCAGGAACG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,42,0,0,0,0,0,0
TTTGTTGTCCAAAGGG-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
gx = Geomux(
    matrix,
    cell_names=matrix.index.values,
    guide_names=matrix.columns.values,
)
gx.test()
assignments = gx.assignments()

In [37]:
assignments

,cell_id,assignment,counts,pvalues,moi,n_umi,min_pvalue,max_count,log_odds,tested
0,AAACCCAAGACGCTCC-1,"[ENSA-4, EPB41L5-2]","[10, 501]","[1.3105544060377932e-05, 8.1585e-320]",2.0,511.0,8.158506e-320,501.0,723.485707,True
1,AAACCCAAGCTAATCC-1,[RELB-4],[515],[8.1585e-320],1.0,515.0,8.158506e-320,515.0,735.667566,True
2,AAACCCAAGGACTGGT-1,[CREB5-2],[9],[8.1585e-320],1.0,9.0,8.158506e-320,9.0,733.081359,True
3,AAACCCAAGGGAGGGT-1,[RPS20-2],[1072],[8.1585e-320],1.0,1072.0,8.158506e-320,1072.0,735.831453,True
4,AAACCCAAGTGCGTCC-1,[NR5A1-2],[155],[8.1585e-320],1.0,155.0,8.158506e-320,155.0,735.667566,True
...,...,...,...,...,...,...,...,...,...,...
25130,TTTGTTGTCAAGGACG-1,"[GLYR1-4, IFITM2-2, NR5A1-2]","[13, 11, 13]","[1.8681828261798048e-15, 1.0667612708672792e-1...",3.0,37.0,1.357556e-25,13.0,22.784792,True
25131,TTTGTTGTCAGGAACG-1,"[RELN-4, ZNF530-2]","[8, 42]","[2.5394439881415008e-17, 4.786580134288002e-83]",2.0,50.0,4.786580e-83,42.0,151.336745,True
25132,TTTGTTGTCCAAAGGG-1,[ENSA-4],[61],[8.1585e-320],1.0,61.0,8.158506e-320,61.0,735.504871,True
25133,TTTGTTGTCCCTCATG-1,[ZNF530-2],[141],[8.1585e-320],1.0,141.0,8.158506e-320,141.0,735.667566,True
